In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Load the dataset
try:
    data = pd.read_csv('Euphoria.csv')
except FileNotFoundError:
    print("File not found. Please check the filename and path.")
    exit()

# Checking and cleaning data if necessary
data.info()
print(data.head())

# Drop or fill NaN values if any (modify as per dataset requirement)
# data.dropna(inplace=True)  # or data.fillna(method='ffill', inplace=True)

# Split features and target variable
X = data.drop(columns=['Name'])
y = data['Name']

# Introduce more significant random noise to the data
X = X + np.random.normal(0, 1, X.shape)  # Adding random noise to features with higher magnitude

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder on the entire dataset
label_encoder.fit(y)

# Transform the target variable using the updated label encoder
y_encoded = label_encoder.transform(y)

# Convert target variable to categorical
num_classes = len(label_encoder.classes_)
y_categorical = to_categorical(y_encoded)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Samples             150 non-null    int64  
 1   Leaves  Length(cm)  150 non-null    float64
 2   Pinnae(pairs)       150 non-null    int64  
 3   Pinnae Length(mm)   150 non-null    float64
 4   Leaflets no         150 non-null    int64  
 5   Leaflets long(mm)   150 non-null    float64
 6   Pods(cm)            150 non-null    float64
 7   Leaflets width(mm)  150 non-null    float64
 8   Name                150 non-null    object 
dtypes: float64(5), int64(3), object(1)
memory usage: 10.7+ KB
   Samples  Leaves  Length(cm)  Pinnae(pairs)  Pinnae Length(mm)  Leaflets no  \
0        1               30.00              8                8.0           10   
1        2               27.30             10                7.0            7   
2        3               24.40              6      

In [ ]:
# prompt: reduce the accuracy of above code

To reduce the accuracy of the code, you can make the following changes:

1. **Reduce the number of epochs**:

